In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy
import codecs

In [3]:
text = 'Icelandic - English.txt'

In [4]:
with codecs.open(text, 'rb', encoding='utf-8',errors='ignore') as f:
    lines = f.read().split('\n')

In [5]:
eng_texts = [] #İngilizce sözcükler için
icelan_texts = [] #İzlanda sözcükler için 

eng_characters = set()
icelan_characters = set()

for line in lines[: min(10000, len(lines) - 1)]:
    
    eng_text= line.split('\u202f')[0].split('\t')[0]
    
    icelan_text= line.split('\u202f')[0].split('\t')[1]
    icelan_text = '\t' + icelan_text + '\n'
    
    eng_texts.append(eng_text)
    
    icelan_texts.append(icelan_text)
    
    for char in eng_text:
        if char not in eng_characters:
            eng_characters.add(char)
            
    for char in icelan_text:
        if char not in icelan_characters:
            icelan_characters.add(char)
            
print("English text :",len(eng_texts))
print("Icelandic text :",len(icelan_texts))

English text : 6488
Icelandic text : 6488


In [6]:
eng_characters = sorted(list(eng_characters)) #Kelime veya cümle içinde geçen karakterler.Alfabedeki harfler,noktalama işaretleri
icelan_characters = sorted(list(icelan_characters))

num_encoder_tokens = len(eng_characters)
print('Number of unique input tokens:', num_encoder_tokens)

num_decoder_tokens = len(icelan_characters)
print('Number of unique output tokens:', num_decoder_tokens)

max_encoder_seq_length = max([len(txt) for txt in eng_texts])
print('Max sequence length for inputs:', max_encoder_seq_length)

max_decoder_seq_length = max([len(txt) for txt in icelan_texts])
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of unique input tokens: 73
Number of unique output tokens: 91
Max sequence length for inputs: 194
Max sequence length for outputs: 211


In [7]:
eng_token_index = dict([(char, i) for i, char in enumerate(eng_characters)])

icelan_token_index = dict([(char, i) for i, char in enumerate(icelan_characters)])

In [8]:
encoder_eng_data = numpy.zeros((len(eng_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')

decoder_eng_data = numpy.zeros((len(eng_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

decoder_icelan_data = numpy.zeros((len(eng_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [9]:
for i, (eng_text, icelan_text) in enumerate(zip(eng_texts, icelan_texts)):
    
    for t, char in enumerate(eng_text):
        encoder_eng_data[i, t, eng_token_index[char]] = 1.
        
    encoder_eng_data[i, t + 1:, eng_token_index[' ']] = 1.
    
    for t, char in enumerate(icelan_text):
        decoder_eng_data[i, t, icelan_token_index[char]] = 1.
        
        if t > 0:
            decoder_icelan_data[i, t - 1, icelan_token_index[char]] = 1.
            
    decoder_eng_data[i, t + 1:, icelan_token_index[' ']] = 1.
    
    decoder_icelan_data[i, t:, icelan_token_index[' ']] = 1.

In [10]:
print(encoder_eng_data.shape)
print(decoder_eng_data.shape)
print(decoder_icelan_data.shape)

(6488, 194, 73)
(6488, 211, 91)
(6488, 211, 91)


In [11]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=64,
          epochs=100,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 31s 214ms/step - loss: 1.5825 - accuracy: 0.6823 - val_loss: 1.1358 - val_accuracy: 0.6915
Epoch 2/100
125/125 [==============================] - 24s 192ms/step - loss: 0.9013 - accuracy: 0.7545 - val_loss: 0.8556 - val_accuracy: 0.7613
Epoch 3/100
125/125 [==============================] - 20s 161ms/step - loss: 0.6999 - accuracy: 0.8044 - val_loss: 0.7397 - val_accuracy: 0.7850
Epoch 4/100
125/125 [==============================] - 20s 162ms/step - loss: 0.6067 - accuracy: 0.8239 - val_loss: 0.6816 - val_accuracy: 0.8003
Epoch 5/100
125/125 [==============================] - 21s 171ms/step - loss: 0.5520 - accuracy: 0.8381 - val_loss: 0.6375 - val_accuracy: 0.8122
Epoch 6/100
125/125 [==============================] - 22s 176ms/step - loss: 0.5121 - accuracy: 0.8497 - val_loss: 0.6026 - val_accuracy: 0.8213
Epoch 7/100
125/125 [==============================] - 22s 174ms/step - loss: 0.4848 - accuracy: 0.8568 - val_loss: 0.5794 -

Epoch 57/100
125/125 [==============================] - 25s 204ms/step - loss: 0.0942 - accuracy: 0.9711 - val_loss: 0.6132 - val_accuracy: 0.8666
Epoch 58/100
125/125 [==============================] - 25s 197ms/step - loss: 0.0925 - accuracy: 0.9715 - val_loss: 0.6183 - val_accuracy: 0.8666
Epoch 59/100
125/125 [==============================] - 25s 202ms/step - loss: 0.0898 - accuracy: 0.9722 - val_loss: 0.6262 - val_accuracy: 0.8661
Epoch 60/100
125/125 [==============================] - 22s 174ms/step - loss: 0.0872 - accuracy: 0.9730 - val_loss: 0.6285 - val_accuracy: 0.8666
Epoch 61/100
125/125 [==============================] - 23s 188ms/step - loss: 0.0866 - accuracy: 0.9730 - val_loss: 0.6310 - val_accuracy: 0.8672
Epoch 62/100
125/125 [==============================] - 25s 198ms/step - loss: 0.0846 - accuracy: 0.9735 - val_loss: 0.6383 - val_accuracy: 0.8662
Epoch 63/100
125/125 [==============================] - 26s 204ms/step - loss: 0.0820 - accuracy: 0.9747 - val_loss: 0

In [12]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [13]:
reverse_input_char_index = dict(
    (i, char) for char, i in eng_token_index.items())

reverse_target_char_index = dict(
    (i, char) for char, i in icelan_token_index.items())

In [14]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, icelan_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [15]:
for seq_index in range(100):
    
    input_seq = encoder_eng_data[seq_index: seq_index + 1]
    
    decoded_sentence = decode_sequence(input_seq)
    
    print('-')
    print('Input sentence:', eng_texts[seq_index])
    
    print('Output sentence:',icelan_texts[seq_index])

-
Input sentence: Hi.
Output sentence: 	Hæ.

-
Input sentence: Hi.
Output sentence: 	Halló.

-
Input sentence: Run!
Output sentence: 	Hlauptu!

-
Input sentence: Run!
Output sentence: 	Hlaupið!

-
Input sentence: Hello!
Output sentence: 	Halló.

-
Input sentence: Cheers!
Output sentence: 	Skál!

-
Input sentence: Got it?
Output sentence: 	Skilið?

-
Input sentence: He ran.
Output sentence: 	Hann hljóp.

-
Input sentence: I know.
Output sentence: 	Ég veit.

-
Input sentence: I'm OK.
Output sentence: 	Það er allt í lagi með mig.

-
Input sentence: Really?
Output sentence: 	Virkilega?

-
Input sentence: Really?
Output sentence: 	Er það satt?

-
Input sentence: Really?
Output sentence: 	Í alvöru?

-
Input sentence: We try.
Output sentence: 	Við reynum.

-
Input sentence: Why me?
Output sentence: 	Af hverju ég?

-
Input sentence: Awesome!
Output sentence: 	Frábært!

-
Input sentence: Get out!
Output sentence: 	Komdu þér út!

-
Input sentence: Get out!
Output sentence: 	Drullaðu þér út!

-
I